In [45]:
# Import Python packages
import pandas as pd
import plotly.express as px
import json
import sys
import cachetools

# Import Snowflake modules
from snowflake.snowpark import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark import Window

In [46]:
# Importing packages
import pandas as pd
import numpy as np

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression


# Feature engine
from feature_engine.discretisation import EqualFrequencyDiscretiser
from feature_engine.encoding import OrdinalEncoder
import feature_engine.imputation as mdi
from feature_engine.encoding import OrdinalEncoder
from feature_engine.discretisation import EqualFrequencyDiscretiser
from feature_engine.transformation import YeoJohnsonTransformer
from feature_engine.encoding import OneHotEncoder
from sklearn.preprocessing import KBinsDiscretizer
from feature_engine.discretisation import EqualWidthDiscretiser
from feature_engine.transformation import ReciprocalTransformer

#Visual
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

import random
import statsmodels.api as sm
from sklearn import metrics, tree
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV
from math import sqrt

In [47]:
# Get account credentials from a json file
with open("data_scientist_auth.json") as f:
    data = json.load(f)
    username = data["username"]
    password = data["password"]
    account = data["account"]

# Specify connection parameters
connection_parameters = {
    "account": account,
    "user": username,
    "password": password,
    "role": "ACCOUNTADMIN",
    "warehouse": "COMPUTE_WH",
    "database": "test_tastybyte",
    "schema": "PUBLIC",
}

# Create Snowpark session
session = Session.builder.configs(connection_parameters).create()

snowpark_df = session.table("GROUP5_DATABASE.PUBLIC.SHIFT11")

In [48]:
snowpark_df.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SHIFT_ID"  |"MENU_ITEM_ID"  |"LOCATION_ID"  |"TRUCK_ID"  |"DATE"      |"TIME"    |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"SHIFT_SALES"  |"QTY_SOLD"  |"TOTAL_SALE_PER_ITEM_PER_SHIFT"  |"SHIFT_NUMBER"  |"CITY"  |"LOCATION"                                         |"ITEM_CATEGORY"  |"ITEM_SUBCATEGORY"  |"SUBCATEGORY"  |"PLACEKEY"   

In [49]:
# Put into Pandas Dataframe 
data = snowpark_df.toPandas()
data.head()

,SHIFT_ID,MENU_ITEM_ID,LOCATION_ID,TRUCK_ID,DATE,TIME,SHIFT_START_TIME,SHIFT_END_TIME,SHIFT_SALES,QTY_SOLD,...,MENU_ITEM_NAME,AVG_TEMPERATURE_AIR_2M_F,AVG_TEMPERATURE_WETBULB_2M_F,AVG_TEMPERATURE_DEWPOINT_2M_F,AVG_TEMPERATURE_WINDCHILL_2M_F,AVG_WIND_SPEED_100M_MPH,SEASON,COG_PER_ITEM_USD,TOTAL_COG_QTY_USD,ITEM_PRICE
0,200570457,36,15295.0,48,2021-06-09,19:30:00,15:30:00,22:30:00,159.47,1,...,Ice Tea,79.9,71.0,66.2,79.9,9.2,SUMMER,0.75,0.75,3.00
1,200570484,19,3331.0,46,2021-06-11,11:07:33,08:00:00,14:00:00,132.26,1,...,Popsicle,59.1,55.1,51.7,58.4,7.1,SUMMER,0.50,0.50,3.00
2,200570531,153,15374.0,60,2021-06-13,19:06:10,15:30:00,22:30:00,256.17,1,...,Hot Ham & Cheese,69.1,61.3,55.8,69.0,12.3,SUMMER,7.00,7.00,11.00
3,200570573,53,3514.0,50,2021-06-16,17:27:43,15:00:00,22:00:00,330.38,3,...,Tonkotsu Ramen,65.8,57.0,48.8,65.8,12.4,SUMMER,7.00,21.00,17.25
4,200570573,56,3514.0,50,2021-06-16,19:26:10,15:00:00,22:00:00,330.38,1,...,Ice Tea,65.8,57.0,48.8,65.8,12.4,SUMMER,0.75,0.75,3.00


In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3047777 entries, 0 to 3047776
Data columns (total 32 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   SHIFT_ID                        int32  
 1   MENU_ITEM_ID                    int16  
 2   LOCATION_ID                     float64
 3   TRUCK_ID                        int16  
 4   DATE                            object 
 5   TIME                            object 
 6   SHIFT_START_TIME                object 
 7   SHIFT_END_TIME                  object 
 8   SHIFT_SALES                     float64
 9   QTY_SOLD                        int64  
 10  TOTAL_SALE_PER_ITEM_PER_SHIFT   float64
 11  SHIFT_NUMBER                    int8   
 12  CITY                            object 
 13  LOCATION                        object 
 14  ITEM_CATEGORY                   object 
 15  ITEM_SUBCATEGORY                object 
 16  SUBCATEGORY                     int8   
 17  PLACEKEY                   

In [51]:
# Check for null values
data.isna().sum()

SHIFT_ID                          0
MENU_ITEM_ID                      0
LOCATION_ID                       0
TRUCK_ID                          0
DATE                              0
TIME                              0
SHIFT_START_TIME                  0
SHIFT_END_TIME                    0
SHIFT_SALES                       0
QTY_SOLD                          0
TOTAL_SALE_PER_ITEM_PER_SHIFT     0
SHIFT_NUMBER                      0
CITY                              0
LOCATION                          0
ITEM_CATEGORY                     0
ITEM_SUBCATEGORY                  0
SUBCATEGORY                       0
PLACEKEY                          0
POSTAL_CODE                       0
MENU_TYPE_ID                      0
MENU_TYPE                         0
TRUCK_BRAND_NAME                  0
MENU_ITEM_NAME                    0
AVG_TEMPERATURE_AIR_2M_F          0
AVG_TEMPERATURE_WETBULB_2M_F      0
AVG_TEMPERATURE_DEWPOINT_2M_F     0
AVG_TEMPERATURE_WINDCHILL_2M_F    0
AVG_WIND_SPEED_100M_MPH     

### Create Bundle and Bundle Price Column

In [52]:
# shift_main = []
# for index, row in data.iterrows():
#     if row['ITEM_CATEGORY'] == "Main":
#         if row['SHIFT_ID'] not in shift_main:
#             shift_main.append(row['SHIFT_ID'])


In [53]:
# shift_main

In [54]:
# with open("output.txt", "w") as file:
#     for item in shift_main:
#         file.write(str(item) + "\n")

In [55]:
# Read the contents of the file into a list
with open("output.txt", "r") as file:
    my_list = file.readlines()

# Strip newline characters from each line and convert to integers (if needed)
my_list = [int(line.strip()) for line in my_list]

# Print the list to verify
print(my_list)

[200524164, 200524178, 200516530, 200524184, 200524192, 200524176, 200516529, 200524181, 200516528, 200570464, 200524169, 200524202, 200524194, 200524959, 200525048, 200524946, 200524960, 200570615, 200524994, 200581454, 200525077, 200524979, 200570739, 200524951, 200581393, 200581417, 200570930, 200549935, 200524997, 200525001, 200525002, 200581469, 200524995, 200516534, 200516531, 200524953, 200570573, 200570640, 200570697, 200524972, 200524977, 200570610, 200570735, 200570763, 200571088, 200571426, 200571698, 200570457, 200570764, 200581398, 200581411, 200571038, 200571548, 200524990, 200525005, 200525009, 200581428, 200581458, 200581459, 200581473, 200525018, 200550691, 200550821, 200571647, 200549955, 200524971, 200524991, 200581456, 200570929, 200571157, 200524165, 200524179, 200524950, 200571012, 200524949, 200581397, 200581399, 200581403, 200581409, 200581402, 200581413, 200571248, 200524973, 200525000, 200525003, 200525004, 200581432, 200581433, 200581452, 200581463, 200581476

In [56]:
# shift sales that have mains
filtered_data = data[data['SHIFT_ID'].isin(my_list)]

In [57]:
filtered_data

,SHIFT_ID,MENU_ITEM_ID,LOCATION_ID,TRUCK_ID,DATE,TIME,SHIFT_START_TIME,SHIFT_END_TIME,SHIFT_SALES,QTY_SOLD,...,MENU_ITEM_NAME,AVG_TEMPERATURE_AIR_2M_F,AVG_TEMPERATURE_WETBULB_2M_F,AVG_TEMPERATURE_DEWPOINT_2M_F,AVG_TEMPERATURE_WINDCHILL_2M_F,AVG_WIND_SPEED_100M_MPH,SEASON,COG_PER_ITEM_USD,TOTAL_COG_QTY_USD,ITEM_PRICE
0,200570457,36,15295.0,48,2021-06-09,19:30:00,15:30:00,22:30:00,159.47,1,...,Ice Tea,79.9,71.0,66.2,79.9,9.2,SUMMER,0.75,0.75,3.00
2,200570531,153,15374.0,60,2021-06-13,19:06:10,15:30:00,22:30:00,256.17,1,...,Hot Ham & Cheese,69.1,61.3,55.8,69.0,12.3,SUMMER,7.00,7.00,11.00
3,200570573,53,3514.0,50,2021-06-16,17:27:43,15:00:00,22:00:00,330.38,3,...,Tonkotsu Ramen,65.8,57.0,48.8,65.8,12.4,SUMMER,7.00,21.00,17.25
4,200570573,56,3514.0,50,2021-06-16,19:26:10,15:00:00,22:00:00,330.38,1,...,Ice Tea,65.8,57.0,48.8,65.8,12.4,SUMMER,0.75,0.75,3.00
5,200570573,53,3514.0,50,2021-06-16,17:28:53,15:00:00,22:00:00,330.38,1,...,Tonkotsu Ramen,65.8,57.0,48.8,65.8,12.4,SUMMER,7.00,7.00,17.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3047772,200521343,102,5216.0,70,2022-06-10,18:36:42,15:30:00,22:30:00,1110.45,4,...,Chicago Dog,72.4,60.1,49.9,72.4,9.1,SUMMER,4.00,16.00,9.00
3047773,200521344,112,1139.0,71,2022-06-10,09:27:34,08:30:00,14:30:00,835.08,1,...,Mothers Favorite,72.1,60.1,50.2,72.1,9.0,SUMMER,4.50,4.50,12.00
3047774,200521345,113,9643.0,71,2022-06-10,22:00:29,15:30:00,22:30:00,1003.22,2,...,The Kitchen Sink,71.9,59.9,49.9,71.9,8.9,SUMMER,6.00,12.00,14.00
3047775,200521348,134,1870.0,73,2022-06-10,11:12:30,08:00:00,14:00:00,666.95,1,...,Bottled Water,72.5,60.1,49.6,72.5,9.3,SUMMER,0.50,0.50,2.00


In [58]:
# # Create a copy of the filtered_data to avoid chained indexing
# filtered_data_copy = filtered_data.copy()

# # Initialize the 'VALUE' column with 0
# filtered_data_copy['VALUE'] = 0

# # Group the data by SHIFT_ID
# grouped_data = filtered_data_copy.groupby('SHIFT_ID')
# i = 1
# # Iterate through each group
# for shift_id, group in grouped_data:
#     mains_data = group[group['ITEM_CATEGORY'] == 'Main']
#     non_mains_data = group[group['ITEM_CATEGORY'] != 'Main']
    
#     # Find highest sales for mains and lowest sales for non-mains
#     highest_sales_mains = mains_data['TOTAL_SALE_PER_ITEM_PER_SHIFT'].max()
#     lowest_sales_non_mains = non_mains_data['TOTAL_SALE_PER_ITEM_PER_SHIFT'].min()
    
#     # Set VALUE to 1 for the row with the highest sales for mains
#     filtered_data_copy.loc[(filtered_data_copy['SHIFT_ID'] == shift_id) & (filtered_data_copy['TOTAL_SALE_PER_ITEM_PER_SHIFT'] == highest_sales_mains), 'VALUE'] = i
    
#     # Set VALUE to 1 for the row with the lowest sales for non-mains
#     filtered_data_copy.loc[(filtered_data_copy['SHIFT_ID'] == shift_id) & (filtered_data_copy['TOTAL_SALE_PER_ITEM_PER_SHIFT'] == lowest_sales_non_mains) & (filtered_data_copy['ITEM_CATEGORY'] != 'Main'), 'VALUE'] = i
#     i += 1

# filtered_data_copy

In [59]:
# file_name = 'filtered_data_final.csv'  # Replace 'output_data.csv' with the desired file name
# filtered_data_copy.to_csv(file_name, index=False)

### Start to run code from here 

In [60]:
file_path = "filtered_data_final.csv"

# Load the CSV into a DataFrame
loaddata = pd.read_csv(file_path)

# Now, you can work with the DataFrame, e.g., print the first few rows
loaddata.head()

,SHIFT_ID,MENU_ITEM_ID,LOCATION_ID,TRUCK_ID,DATE,TIME,SHIFT_START_TIME,SHIFT_END_TIME,SHIFT_SALES,QTY_SOLD,...,AVG_TEMPERATURE_AIR_2M_F,AVG_TEMPERATURE_WETBULB_2M_F,AVG_TEMPERATURE_DEWPOINT_2M_F,AVG_TEMPERATURE_WINDCHILL_2M_F,AVG_WIND_SPEED_100M_MPH,SEASON,COG_PER_ITEM_USD,TOTAL_COG_QTY_USD,ITEM_PRICE,VALUE
0,200552443,121,1260.0,42,2021-07-13,18:36:49,15:30:00,22:30:00,219.27,2,...,65.3,59.3,55.2,65.1,4.0,SUMMER,12.0,24.0,20.0,0
1,200524171,142,15126.0,74,2022-09-12,17:47:39,15:30:00,22:30:00,1744.51,3,...,73.8,70.2,68.6,73.8,4.3,AUTUMN,11.0,33.0,18.0,0
2,200552177,115,2119.0,41,2021-07-01,20:08:58,15:30:00,22:30:00,249.16,1,...,68.1,62.3,58.7,68.1,3.0,SUMMER,0.5,0.5,3.0,0
3,200552486,124,1917.0,42,2021-07-15,14:03:28,08:30:00,14:30:00,217.09,1,...,62.7,57.4,53.6,62.2,4.5,SUMMER,0.5,0.5,2.0,26588
4,200552618,125,3678.0,42,2021-07-21,09:18:19,08:30:00,14:30:00,197.14,1,...,60.6,55.4,51.3,60.5,3.2,SUMMER,0.5,0.5,3.0,0


In [61]:
loaddata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2859478 entries, 0 to 2859477
Data columns (total 33 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   SHIFT_ID                        int64  
 1   MENU_ITEM_ID                    int64  
 2   LOCATION_ID                     float64
 3   TRUCK_ID                        int64  
 4   DATE                            object 
 5   TIME                            object 
 6   SHIFT_START_TIME                object 
 7   SHIFT_END_TIME                  object 
 8   SHIFT_SALES                     float64
 9   QTY_SOLD                        int64  
 10  TOTAL_SALE_PER_ITEM_PER_SHIFT   float64
 11  SHIFT_NUMBER                    int64  
 12  CITY                            object 
 13  LOCATION                        object 
 14  ITEM_CATEGORY                   object 
 15  ITEM_SUBCATEGORY                object 
 16  SUBCATEGORY                     int64  
 17  PLACEKEY                   

In [62]:
# add back the rows of shift that have no mains 
without_mains = data[~data['SHIFT_ID'].isin(my_list)]
without_mains

,SHIFT_ID,MENU_ITEM_ID,LOCATION_ID,TRUCK_ID,DATE,TIME,SHIFT_START_TIME,SHIFT_END_TIME,SHIFT_SALES,QTY_SOLD,...,MENU_ITEM_NAME,AVG_TEMPERATURE_AIR_2M_F,AVG_TEMPERATURE_WETBULB_2M_F,AVG_TEMPERATURE_DEWPOINT_2M_F,AVG_TEMPERATURE_WINDCHILL_2M_F,AVG_WIND_SPEED_100M_MPH,SEASON,COG_PER_ITEM_USD,TOTAL_COG_QTY_USD,ITEM_PRICE
1,200570484,19,3331.0,46,2021-06-11,11:07:33,08:00:00,14:00:00,132.26,1,...,Popsicle,59.1,55.1,51.7,58.4,7.1,SUMMER,0.50,0.50,3.0
13,200570613,18,15280.0,46,2021-06-19,20:22:08,15:30:00,22:30:00,372.96,1,...,Mango Sticky Rice,75.6,66.8,62.0,75.6,14.8,SUMMER,1.25,1.25,5.0
15,200570612,11,15419.0,46,2021-06-19,13:12:43,08:30:00,14:30:00,265.42,1,...,Sugar Cone,76.5,67.0,61.8,76.5,15.2,SUMMER,2.50,2.50,6.0
20,200570613,14,15280.0,46,2021-06-19,16:42:34,15:30:00,22:30:00,372.96,1,...,Bottled Water,75.6,66.8,62.0,75.6,14.8,SUMMER,0.50,0.50,2.0
24,200570613,17,15280.0,46,2021-06-19,19:52:15,15:30:00,22:30:00,372.96,1,...,Ice Cream Sandwich,75.6,66.8,62.0,75.6,14.8,SUMMER,1.00,1.00,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3047697,200521234,12,2131.0,61,2022-06-07,12:39:39,08:30:00,14:30:00,678.74,2,...,Waffle Cone,70.1,63.2,58.8,70.1,18.0,SUMMER,2.50,5.00,6.0
3047744,200521295,11,15182.0,61,2022-06-09,16:08:38,15:00:00,22:00:00,438.00,1,...,Sugar Cone,74.6,66.7,61.8,74.6,16.4,SUMMER,2.50,2.50,6.0
3047745,200521295,14,15182.0,61,2022-06-09,17:30:16,15:00:00,22:00:00,438.00,1,...,Bottled Water,74.6,66.7,61.8,74.6,16.4,SUMMER,0.50,0.50,2.0
3047756,200521324,13,2806.0,61,2022-06-10,09:37:10,08:00:00,14:00:00,357.66,1,...,Two Scoop Bowl,72.3,60.1,50.0,72.3,9.1,SUMMER,3.00,3.00,7.0


In [63]:
without_mains['VALUE'] = 0
without_mains

C:\Users\kohyu\AppData\Local\Temp\ipykernel_22916\3949425381.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  without_mains['VALUE'] = 0


,SHIFT_ID,MENU_ITEM_ID,LOCATION_ID,TRUCK_ID,DATE,TIME,SHIFT_START_TIME,SHIFT_END_TIME,SHIFT_SALES,QTY_SOLD,...,AVG_TEMPERATURE_AIR_2M_F,AVG_TEMPERATURE_WETBULB_2M_F,AVG_TEMPERATURE_DEWPOINT_2M_F,AVG_TEMPERATURE_WINDCHILL_2M_F,AVG_WIND_SPEED_100M_MPH,SEASON,COG_PER_ITEM_USD,TOTAL_COG_QTY_USD,ITEM_PRICE,VALUE
1,200570484,19,3331.0,46,2021-06-11,11:07:33,08:00:00,14:00:00,132.26,1,...,59.1,55.1,51.7,58.4,7.1,SUMMER,0.50,0.50,3.0,0
13,200570613,18,15280.0,46,2021-06-19,20:22:08,15:30:00,22:30:00,372.96,1,...,75.6,66.8,62.0,75.6,14.8,SUMMER,1.25,1.25,5.0,0
15,200570612,11,15419.0,46,2021-06-19,13:12:43,08:30:00,14:30:00,265.42,1,...,76.5,67.0,61.8,76.5,15.2,SUMMER,2.50,2.50,6.0,0
20,200570613,14,15280.0,46,2021-06-19,16:42:34,15:30:00,22:30:00,372.96,1,...,75.6,66.8,62.0,75.6,14.8,SUMMER,0.50,0.50,2.0,0
24,200570613,17,15280.0,46,2021-06-19,19:52:15,15:30:00,22:30:00,372.96,1,...,75.6,66.8,62.0,75.6,14.8,SUMMER,1.00,1.00,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3047697,200521234,12,2131.0,61,2022-06-07,12:39:39,08:30:00,14:30:00,678.74,2,...,70.1,63.2,58.8,70.1,18.0,SUMMER,2.50,5.00,6.0,0
3047744,200521295,11,15182.0,61,2022-06-09,16:08:38,15:00:00,22:00:00,438.00,1,...,74.6,66.7,61.8,74.6,16.4,SUMMER,2.50,2.50,6.0,0
3047745,200521295,14,15182.0,61,2022-06-09,17:30:16,15:00:00,22:00:00,438.00,1,...,74.6,66.7,61.8,74.6,16.4,SUMMER,0.50,0.50,2.0,0
3047756,200521324,13,2806.0,61,2022-06-10,09:37:10,08:00:00,14:00:00,357.66,1,...,72.3,60.1,50.0,72.3,9.1,SUMMER,3.00,3.00,7.0,0


In [64]:
# Concatenate the two DataFrames along the rows (axis=0)
combined_table = pd.concat([loaddata, without_mains], ignore_index=True)

In [65]:
# Total Price of bundle -10%
def calculate_discounted_sales(row):
    if row['VALUE'] != 0:
        return row['ITEM_PRICE'] * 0.9
    return row['ITEM_PRICE']

combined_table['discount_10%'] = combined_table.apply(calculate_discounted_sales, axis=1)

In [66]:
combined_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3047777 entries, 0 to 3047776
Data columns (total 34 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   SHIFT_ID                        int64  
 1   MENU_ITEM_ID                    int64  
 2   LOCATION_ID                     float64
 3   TRUCK_ID                        int64  
 4   DATE                            object 
 5   TIME                            object 
 6   SHIFT_START_TIME                object 
 7   SHIFT_END_TIME                  object 
 8   SHIFT_SALES                     float64
 9   QTY_SOLD                        int64  
 10  TOTAL_SALE_PER_ITEM_PER_SHIFT   float64
 11  SHIFT_NUMBER                    int64  
 12  CITY                            object 
 13  LOCATION                        object 
 14  ITEM_CATEGORY                   object 
 15  ITEM_SUBCATEGORY                object 
 16  SUBCATEGORY                     int64  
 17  PLACEKEY                   

In [67]:
# check value
filtereds_df = combined_table[combined_table['VALUE'] != 0]
filtereds_df.head(20) 

,SHIFT_ID,MENU_ITEM_ID,LOCATION_ID,TRUCK_ID,DATE,TIME,SHIFT_START_TIME,SHIFT_END_TIME,SHIFT_SALES,QTY_SOLD,...,AVG_TEMPERATURE_WETBULB_2M_F,AVG_TEMPERATURE_DEWPOINT_2M_F,AVG_TEMPERATURE_WINDCHILL_2M_F,AVG_WIND_SPEED_100M_MPH,SEASON,COG_PER_ITEM_USD,TOTAL_COG_QTY_USD,ITEM_PRICE,VALUE,discount_10%
3,200552486,124,1917.0,42,2021-07-15,14:03:28,08:30:00,14:30:00,217.09,1,...,57.4,53.6,62.2,4.5,SUMMER,0.50,0.50,2.0,26588,1.80
5,200524180,44,2727.0,64,2022-09-13,10:39:23,08:00:00,14:00:00,807.14,1,...,69.3,65.0,76.4,8.5,AUTUMN,0.50,0.50,2.0,16893,1.80
18,200329606,104,3877.0,10,2022-03-31,13:47:44,08:00:00,14:00:00,75.64,1,...,49.2,45.4,52.3,6.7,SPRING,0.50,0.50,2.0,3818,1.80
20,200549935,124,15089.0,42,2021-01-26,18:39:02,15:00:00,22:00:00,476.59,1,...,36.5,27.1,37.6,8.8,WINTER,0.50,0.50,2.0,26476,1.80
52,200531408,84,14853.0,23,2021-03-24,12:00:16,08:00:00,14:00:00,417.12,1,...,31.2,25.9,29.9,8.6,SPRING,0.50,0.50,2.0,18301,1.80
85,200329579,91,1249.0,9,2022-03-30,20:05:50,15:00:00,22:00:00,65.63,4,...,48.6,45.7,48.5,11.1,SPRING,5.00,20.00,11.0,3793,9.90
87,200329587,132,2254.0,13,2022-03-30,20:07:10,15:00:00,22:00:00,151.28,6,...,48.2,45.6,47.3,11.8,SPRING,5.00,30.00,11.0,3801,9.90
109,200540892,94,14794.0,24,2022-04-15,12:33:49,08:30:00,14:30:00,327.53,1,...,40.5,18.2,49.6,9.0,SPRING,0.50,0.50,2.0,21941,1.80
121,200329616,154,1800.0,15,2022-03-31,12:44:03,08:30:00,14:30:00,85.71,1,...,49.5,45.9,52.5,6.8,SPRING,0.50,0.50,2.0,3828,1.80
124,200514146,74,3274.0,67,2021-08-17,13:23:02,09:00:00,15:00:00,667.44,1,...,69.8,66.7,76.2,9.6,SUMMER,0.50,0.50,2.0,9407,1.80


In [68]:
combined_table['DATE'] = pd.to_datetime(combined_table['DATE'])

In [69]:
# Sort the DataFrame by the 'DATE' column in ascending order
combined_table = combined_table.sort_values(by='DATE')

# Display the sorted DataFrame
print(combined_table)

          SHIFT_ID  MENU_ITEM_ID  LOCATION_ID  TRUCK_ID       DATE      TIME  \
2914861  100309823            18       3051.0         1 2019-01-04  12:42:05   
2869415  100309823            12       3051.0         1 2019-01-04  12:42:05   
2912877  100309823            19       3051.0         1 2019-01-04  12:42:05   
3005201  100309838            17       1058.0         1 2019-01-11  18:06:59   
3038170  100309838            13       1058.0         1 2019-01-11  20:26:02   
...            ...           ...          ...       ...        ...       ...   
1417015  200525656            73       3007.0        67 2022-11-01  10:14:58   
2731649  200525656            73       3007.0        67 2022-11-01  12:56:35   
2027216  200525661            93       3159.0        69 2022-11-01  20:55:34   
2015617  200525659            85      13096.0        68 2022-11-01  18:28:42   
230020   200525661            91       3159.0        69 2022-11-01  17:04:04   

        SHIFT_START_TIME SHIFT_END_TIME

#### Find out data that is not 2022

In [70]:
not_2022_data = combined_table[combined_table['DATE'].dt.year != 2022]

In [71]:
not_2022_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 610327 entries, 2914861 to 1308959
Data columns (total 34 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   SHIFT_ID                        610327 non-null  int64         
 1   MENU_ITEM_ID                    610327 non-null  int64         
 2   LOCATION_ID                     610327 non-null  float64       
 3   TRUCK_ID                        610327 non-null  int64         
 4   DATE                            610327 non-null  datetime64[ns]
 5   TIME                            610327 non-null  object        
 6   SHIFT_START_TIME                610327 non-null  object        
 7   SHIFT_END_TIME                  610327 non-null  object        
 8   SHIFT_SALES                     610327 non-null  float64       
 9   QTY_SOLD                        610327 non-null  int64         
 10  TOTAL_SALE_PER_ITEM_PER_SHIFT   610327 non-null  

In [72]:
winter_2022_data = combined_table[(combined_table['DATE'].dt.year == 2022) & (combined_table['SEASON'] == 'WINTER')]

In [73]:
winter_2022_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 444361 entries, 379278 to 2791318
Data columns (total 34 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   SHIFT_ID                        444361 non-null  int64         
 1   MENU_ITEM_ID                    444361 non-null  int64         
 2   LOCATION_ID                     444361 non-null  float64       
 3   TRUCK_ID                        444361 non-null  int64         
 4   DATE                            444361 non-null  datetime64[ns]
 5   TIME                            444361 non-null  object        
 6   SHIFT_START_TIME                444361 non-null  object        
 7   SHIFT_END_TIME                  444361 non-null  object        
 8   SHIFT_SALES                     444361 non-null  float64       
 9   QTY_SOLD                        444361 non-null  int64         
 10  TOTAL_SALE_PER_ITEM_PER_SHIFT   444361 non-null  f

In [74]:
final_table = pd.concat([not_2022_data, winter_2022_data], ignore_index=True)

In [75]:
final_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1054688 entries, 0 to 1054687
Data columns (total 34 columns):
 #   Column                          Non-Null Count    Dtype         
---  ------                          --------------    -----         
 0   SHIFT_ID                        1054688 non-null  int64         
 1   MENU_ITEM_ID                    1054688 non-null  int64         
 2   LOCATION_ID                     1054688 non-null  float64       
 3   TRUCK_ID                        1054688 non-null  int64         
 4   DATE                            1054688 non-null  datetime64[ns]
 5   TIME                            1054688 non-null  object        
 6   SHIFT_START_TIME                1054688 non-null  object        
 7   SHIFT_END_TIME                  1054688 non-null  object        
 8   SHIFT_SALES                     1054688 non-null  float64       
 9   QTY_SOLD                        1054688 non-null  int64         
 10  TOTAL_SALE_PER_ITEM_PER_SHIFT   1054688 no

In [76]:
# drop columns that are not related 
final_table = final_table.drop(['PLACEKEY', 'POSTAL_CODE', 'SHIFT_START_TIME', 'SHIFT_END_TIME', 'SHIFT_ID', 'TRUCK_ID',
                  'MENU_TYPE_ID', 'ITEM_SUBCATEGORY', 'MENU_ITEM_ID', 'SHIFT_SALES', 'TIME',
                   'QTY_SOLD', 'LOCATION', 'TOTAL_COG_QTY_USD'
                  ], axis = 1)

In [77]:
weather_col = ['AVG_TEMPERATURE_AIR_2M_F',
 'AVG_TEMPERATURE_WETBULB_2M_F',
 'AVG_TEMPERATURE_DEWPOINT_2M_F',
 'AVG_TEMPERATURE_WINDCHILL_2M_F',
 'AVG_WIND_SPEED_100M_MPH']
for wc in weather_col:
    final_table[wc] = final_table[wc].astype(float)

In [78]:
final_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1054688 entries, 0 to 1054687
Data columns (total 20 columns):
 #   Column                          Non-Null Count    Dtype         
---  ------                          --------------    -----         
 0   LOCATION_ID                     1054688 non-null  float64       
 1   DATE                            1054688 non-null  datetime64[ns]
 2   TOTAL_SALE_PER_ITEM_PER_SHIFT   1054688 non-null  float64       
 3   SHIFT_NUMBER                    1054688 non-null  int64         
 4   CITY                            1054688 non-null  object        
 5   ITEM_CATEGORY                   1054688 non-null  object        
 6   SUBCATEGORY                     1054688 non-null  int64         
 7   MENU_TYPE                       1054688 non-null  object        
 8   TRUCK_BRAND_NAME                1054688 non-null  object        
 9   MENU_ITEM_NAME                  1054688 non-null  object        
 10  AVG_TEMPERATURE_AIR_2M_F        1054688 no

In [79]:
cat_col=[col for col in final_table.columns if final_table[col].dtypes=="O" ]
cat_col


['CITY',
 'ITEM_CATEGORY',
 'MENU_TYPE',
 'TRUCK_BRAND_NAME',
 'MENU_ITEM_NAME',
 'SEASON']

In [80]:
num_col=[col for col in final_table.columns if final_table[col].dtypes!="O" ]
num_col

['LOCATION_ID',
 'DATE',
 'TOTAL_SALE_PER_ITEM_PER_SHIFT',
 'SHIFT_NUMBER',
 'SUBCATEGORY',
 'AVG_TEMPERATURE_AIR_2M_F',
 'AVG_TEMPERATURE_WETBULB_2M_F',
 'AVG_TEMPERATURE_DEWPOINT_2M_F',
 'AVG_TEMPERATURE_WINDCHILL_2M_F',
 'AVG_WIND_SPEED_100M_MPH',
 'COG_PER_ITEM_USD',
 'ITEM_PRICE',
 'VALUE',
 'discount_10%']

In [81]:
file_name = 'final_dataset_diff.csv'  # Replace 'output_data.csv' with the desired file name
final_table.to_csv(file_name, index=False)

In [82]:
file_path = "final_dataset_diff.csv"

# Load the CSV into a DataFrame
final_table = pd.read_csv(file_path)

# Now, you can work with the DataFrame, e.g., print the first few rows
final_table.head()

,LOCATION_ID,DATE,TOTAL_SALE_PER_ITEM_PER_SHIFT,SHIFT_NUMBER,CITY,ITEM_CATEGORY,SUBCATEGORY,MENU_TYPE,TRUCK_BRAND_NAME,MENU_ITEM_NAME,AVG_TEMPERATURE_AIR_2M_F,AVG_TEMPERATURE_WETBULB_2M_F,AVG_TEMPERATURE_DEWPOINT_2M_F,AVG_TEMPERATURE_WINDCHILL_2M_F,AVG_WIND_SPEED_100M_MPH,SEASON,COG_PER_ITEM_USD,ITEM_PRICE,VALUE,discount_10%
0,3051.0,2019-01-04,5.0,1,San Mateo,Dessert,1,Ice Cream,Freezing Point,Mango Sticky Rice,47.1,43.4,39.0,46.6,6.5,WINTER,1.25,5.0,0,5.0
1,3051.0,2019-01-04,6.0,1,San Mateo,Dessert,1,Ice Cream,Freezing Point,Waffle Cone,47.1,43.4,39.0,46.6,6.5,WINTER,2.50,6.0,0,6.0
2,3051.0,2019-01-04,3.0,1,San Mateo,Dessert,1,Ice Cream,Freezing Point,Popsicle,47.1,43.4,39.0,46.6,6.5,WINTER,0.50,3.0,0,3.0
3,1058.0,2019-01-11,4.0,2,San Mateo,Dessert,1,Ice Cream,Freezing Point,Ice Cream Sandwich,51.7,50.5,49.5,51.2,12.2,WINTER,1.00,4.0,0,4.0
4,1058.0,2019-01-11,7.0,2,San Mateo,Dessert,1,Ice Cream,Freezing Point,Two Scoop Bowl,51.7,50.5,49.5,51.2,12.2,WINTER,3.00,7.0,0,7.0


## Outlier Check

In [83]:
# #Search for outliers within the Num Variables

# #Create dataframe for Num variables 
# def diagnostic_plots(df, variable):
#     # function takes a dataframe (df) and
#     # the variable of interest as arguments

#     # define figure size
#     plt.figure(figsize=(16, 4))

#     # histogram
#     plt.subplot(1, 3, 1)
#     sns.distplot(df[variable], bins=30)
#     plt.title('Histogram')

#     # Q-Q plot
#     plt.subplot(1, 3, 2)
#     stats.probplot(df[variable], dist="norm", plot=plt)
#     plt.ylabel('RM quantiles')

#     # boxplot
#     plt.subplot(1, 3, 3)
#     sns.boxplot(y=df[variable])
#     plt.title('Boxplot')

#     plt.show()
    
# def diagnostic_plots_numeric(df, variable):
    
#     # function to plot a histogram and a Q-Q plot
#     # side by side, for a certain variable
    
#     plt.figure(figsize=(15,6))
#     plt.subplot(1, 2, 1)
#     df[variable].hist(bins=30)

#     plt.subplot(1, 2, 2)
#     stats.probplot(df[variable], dist="norm", plot=plt)
#     plt.ylabel('RM quantiles')
#     plt.suptitle(variable)

#     plt.show()





# #Checking Amount of Outlier for Variable age,length of service and training score and no of trainings

# diagnostic_plots(combined_table, 'COG_PER_ITEM_USD')
# diagnostic_plots(combined_table, 'TOTAL_SALE_PER_ITEM_PER_SHIFT')
# diagnostic_plots(combined_table, 'AVG_TEMPERATURE_WETBULB_2M_F')
# diagnostic_plots(combined_table, 'AVG_TEMPERATURE_DEWPOINT_2M_F')
# diagnostic_plots(combined_table, 'AVG_TEMPERATURE_WINDCHILL_2M_F')
# diagnostic_plots(combined_table, 'AVG_WIND_SPEED_100M_MPH')
# diagnostic_plots(combined_table, 'AVG_TEMPERATURE_AIR_2M_F')



## One Hot Encoding 

In [84]:
encoder = OneHotEncoder()
encoder.fit(final_table)
df_OHE=encoder.transform(final_table)

df_OHE

c:\Users\kohyu\anaconda3\envs\py38_env\lib\site-packages\feature_engine\encoding\one_hot.py:277: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[f"{feature}_{category}"] = np.where(X[feature] == category, 1, 0)
c:\Users\kohyu\anaconda3\envs\py38_env\lib\site-packages\feature_engine\encoding\one_hot.py:277: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[f"{feature}_{category}"] = np.where(X[feature] == category, 1, 0)
c:\Users\kohyu\anaconda3\envs\py38_env\lib\site-packages\feature_engine\encoding\one_hot.py:277: PerformanceWarn

,LOCATION_ID,DATE,TOTAL_SALE_PER_ITEM_PER_SHIFT,SHIFT_NUMBER,SUBCATEGORY,AVG_TEMPERATURE_AIR_2M_F,AVG_TEMPERATURE_WETBULB_2M_F,AVG_TEMPERATURE_DEWPOINT_2M_F,AVG_TEMPERATURE_WINDCHILL_2M_F,AVG_WIND_SPEED_100M_MPH,...,MENU_ITEM_NAME_Tandoori Mixed Grill,MENU_ITEM_NAME_Lean Chicken Tikka Masala,MENU_ITEM_NAME_Combination Curry,MENU_ITEM_NAME_Buffalo Mac & Cheese,MENU_ITEM_NAME_Standard Mac & Cheese,MENU_ITEM_NAME_Lobster Mac & Cheese,SEASON_WINTER,SEASON_SPRING,SEASON_SUMMER,SEASON_AUTUMN
0,3051.0,2019-01-04,5.0,1,1,47.1,43.4,39.0,46.6,6.5,...,0,0,0,0,0,0,1,0,0,0
1,3051.0,2019-01-04,6.0,1,1,47.1,43.4,39.0,46.6,6.5,...,0,0,0,0,0,0,1,0,0,0
2,3051.0,2019-01-04,3.0,1,1,47.1,43.4,39.0,46.6,6.5,...,0,0,0,0,0,0,1,0,0,0
3,1058.0,2019-01-11,4.0,2,1,51.7,50.5,49.5,51.2,12.2,...,0,0,0,0,0,0,1,0,0,0
4,1058.0,2019-01-11,7.0,2,1,51.7,50.5,49.5,51.2,12.2,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054683,14930.0,2022-02-28,51.0,5,3,46.5,37.5,16.1,44.7,7.0,...,0,1,0,0,0,0,1,0,0,0
1054684,1265.0,2022-02-28,12.0,4,2,51.6,50.9,50.2,49.5,12.2,...,0,0,0,0,0,0,1,0,0,0
1054685,3671.0,2022-02-28,30.0,2,3,51.8,51.1,50.4,49.4,11.9,...,0,0,1,0,0,0,1,0,0,0
1054686,2904.0,2022-02-28,48.0,4,3,50.4,49.9,49.4,48.1,13.2,...,0,0,0,0,0,0,1,0,0,0


In [88]:
df_OHE['DATE'] = pd.to_datetime(df_OHE['DATE'])

In [89]:
not_vaildate = df_OHE[df_OHE['DATE'].dt.year != 2022]
not_vaildate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 610327 entries, 0 to 610326
Columns: 115 entries, LOCATION_ID to SEASON_AUTUMN
dtypes: datetime64[ns](1), float64(10), int32(101), int64(3)
memory usage: 305.0 MB


In [92]:
not_vaildate = not_vaildate.drop(['DATE'], axis=1)

In [91]:
vaildate = df_OHE[df_OHE['DATE'].dt.year == 2022]
vaildate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 444361 entries, 610327 to 1054687
Columns: 115 entries, LOCATION_ID to SEASON_AUTUMN
dtypes: datetime64[ns](1), float64(10), int32(101), int64(3)
memory usage: 222.1 MB


In [93]:
vaildate = vaildate.drop(['DATE'], axis=1)

In [94]:
# For Validation Data
X_holdout = vaildate.drop(['TOTAL_SALE_PER_ITEM_PER_SHIFT'], axis=1)
y_holdout = vaildate['TOTAL_SALE_PER_ITEM_PER_SHIFT']

In [95]:
#for train and testing data 
X_train_test = not_vaildate.drop(['TOTAL_SALE_PER_ITEM_PER_SHIFT'], axis=1)
y_train_test = not_vaildate['TOTAL_SALE_PER_ITEM_PER_SHIFT']

In [96]:
X_train, X_test, y_train, y_test = train_test_split(X_train_test, y_train_test, test_size=0.3, random_state=2)

In [97]:
print(X_train.info())
print(X_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 427228 entries, 508495 to 100879
Columns: 113 entries, LOCATION_ID to SEASON_AUTUMN
dtypes: float64(9), int32(101), int64(3)
memory usage: 207.0 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 183099 entries, 91651 to 219283
Columns: 113 entries, LOCATION_ID to SEASON_AUTUMN
dtypes: float64(9), int32(101), int64(3)
memory usage: 88.7 MB
None


In [98]:
#Standardization
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)
display(X_train_scaled.describe())

,LOCATION_ID,SHIFT_NUMBER,SUBCATEGORY,AVG_TEMPERATURE_AIR_2M_F,AVG_TEMPERATURE_WETBULB_2M_F,AVG_TEMPERATURE_DEWPOINT_2M_F,AVG_TEMPERATURE_WINDCHILL_2M_F,AVG_WIND_SPEED_100M_MPH,COG_PER_ITEM_USD,ITEM_PRICE,...,MENU_ITEM_NAME_Tandoori Mixed Grill,MENU_ITEM_NAME_Lean Chicken Tikka Masala,MENU_ITEM_NAME_Combination Curry,MENU_ITEM_NAME_Buffalo Mac & Cheese,MENU_ITEM_NAME_Standard Mac & Cheese,MENU_ITEM_NAME_Lobster Mac & Cheese,SEASON_WINTER,SEASON_SPRING,SEASON_SUMMER,SEASON_AUTUMN
count,4.272280e+05,4.272280e+05,4.272280e+05,4.272280e+05,4.272280e+05,4.272280e+05,4.272280e+05,4.272280e+05,4.272280e+05,4.272280e+05,...,4.272280e+05,4.272280e+05,4.272280e+05,4.272280e+05,4.272280e+05,4.272280e+05,4.272280e+05,4.272280e+05,4.272280e+05,4.272280e+05
mean,4.961166e-17,5.563225e-18,-1.633210e-17,3.914714e-16,-1.155388e-16,-3.869477e-16,-4.872520e-16,-4.407005e-16,3.255776e-16,-2.178722e-17,...,-2.128827e-17,-4.616063e-17,2.267284e-17,-5.654698e-19,-1.393717e-17,7.627190e-17,-4.079698e-17,3.813387e-17,6.595207e-17,1.212434e-17
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,...,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,-1.071968e+00,-1.619966e+00,-1.241233e+00,-4.369965e+00,-4.381504e+00,-3.175423e+00,-4.393734e+00,-1.699994e+00,-1.194483e+00,-1.415418e+00,...,-1.260319e-01,-1.268059e-01,-1.254749e-01,-1.344339e-01,-1.334379e-01,-1.346322e-01,-6.255183e-01,-2.980691e-01,-3.257879e-01,-1.086130e+00
25%,-7.985059e-01,-1.029460e+00,-1.241233e+00,-6.820629e-01,-7.138941e-01,-7.627117e-01,-7.184396e-01,-7.925709e-01,-9.567367e-01,-8.064820e-01,...,-1.260319e-01,-1.268059e-01,-1.254749e-01,-1.344339e-01,-1.334379e-01,-1.346322e-01,-6.255183e-01,-2.980691e-01,-3.257879e-01,-1.086130e+00
50%,-5.870901e-01,1.515518e-01,-7.830981e-02,-6.305325e-02,-8.352364e-02,1.015430e-01,-3.642610e-02,-2.357433e-01,-8.500037e-02,5.433199e-03,...,-1.260319e-01,-1.268059e-01,-1.254749e-01,-1.344339e-01,-1.334379e-01,-1.346322e-01,-6.255183e-01,-2.980691e-01,-3.257879e-01,9.206998e-01
75%,1.375577e+00,7.420576e-01,1.084614e+00,5.908302e-01,6.901128e-01,7.641383e-01,6.380094e-01,5.891865e-01,5.489897e-01,6.143696e-01,...,-1.260319e-01,-1.268059e-01,-1.254749e-01,-1.344339e-01,-1.334379e-01,-1.346322e-01,1.598674e+00,-2.980691e-01,-3.257879e-01,9.206998e-01
max,1.488355e+00,1.332563e+00,1.084614e+00,3.232519e+00,2.877307e+00,2.499850e+00,2.987167e+00,3.971399e+00,2.450960e+00,2.441179e+00,...,7.934500e+00,7.886068e+00,7.969721e+00,7.438602e+00,7.494124e+00,7.427642e+00,1.598674e+00,3.354927e+00,3.069482e+00,9.206998e-01


In [99]:
# Base Model
model_linear_regression = LinearRegression()
model_linear_regression.fit(X_train, y_train)
y_pred_holdout = model_linear_regression.predict(X_holdout)
mse_holdout = mean_squared_error(y_holdout, y_pred_holdout)

print('train_mse:', mean_squared_error(y_train, model_linear_regression.predict(X_train)), 
      '\n test_mse:', mean_squared_error(y_test, model_linear_regression.predict(X_test)))
print('train_r-sq:', r2_score(y_train, model_linear_regression.predict(X_train)), 
      '\n test_r-sq:', r2_score(y_test, model_linear_regression.predict(X_test)))

print('holdout_mse:', mean_squared_error(y_holdout, model_linear_regression.predict(X_holdout)), 
      '\n holdout-sq:',  r2_score(y_holdout, model_linear_regression.predict(X_holdout)))

train_mse: 63.70941166983502 
 test_mse: 64.03223109555096
train_r-sq: 0.5833938377624983 
 test_r-sq: 0.5815537619782661
holdout_mse: 69.41113465583148 
 holdout-sq: 0.5814686425840497


In [100]:
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor, VotingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor 
from sklearn.neighbors import KNeighborsRegressor

#### Decision Tree Regressor 

In [105]:
dtreg = DecisionTreeRegressor()
dtreg.fit(X_train, y_train)

dtreg_train_mse = mean_squared_error(dtreg.predict(X_train), y_train)
print('Decision Tree Training MSE:', dtreg_train_mse)
dtreg_train_R2 = dtreg.score(X_train, y_train)
print("Decision Tree Training R2:", dtreg_train_R2)

dtreg_test_mse = mean_squared_error(dtreg.predict(X_test), y_test)
print('\nDecision Tree Testing MSE:',dtreg_test_mse)
dtreg_test_R2 = dtreg.score(X_test, y_test)
print('Decision Tree Testing R2: ', dtreg_test_R2)

dtreg_test_mse = mean_squared_error(dtreg.predict(X_holdout), y_holdout)
print('\nDecision Tree Testing MSE:',dtreg_test_mse)
dtreg_test_R2 = dtreg.score(X_holdout, y_holdout)
print('Decision Tree Testing R2: ', dtreg_test_R2)

Decision Tree Training MSE: 52.69899733267738
Decision Tree Training R2: 0.6553927205244288

Decision Tree Testing MSE: 69.27318502564987
Decision Tree Testing R2:  0.5473044875398578

Decision Tree Testing MSE: 84.61609070628995
Decision Tree Testing R2:  0.489786653421903


In [110]:
tree_reg_gs = DecisionTreeRegressor()
# Use GridSearch to find the best combination of model hyperparameters
param_grid = { "criterion" : ["absolute_error", "squared_error"],
              "min_samples_leaf" : [1, 5, 10], 
              "min_samples_split" : [2, 4, 10, 12, 16], 
              'max_depth' : [2,3,4,5,6,7]}

gs = GridSearchCV(tree_reg_gs, param_grid=param_grid, scoring='neg_mean_squared_error', cv= 5, n_jobs=-1)
# cv: number of partitions for cross validation
# n_jobs: number of jobs to run in parallel, -1 means using all processors

gs = gs.fit(X_train, y_train) # 

print(gs.best_score_)
print(gs.best_params_)

KeyboardInterrupt: 

##### MLP Regressor

In [106]:
mlpr = MLPRegressor()
mlpr.fit(X_train,y_train)
train_mse = mean_squared_error(mlpr.predict(X_train), y_train)
test_mse = mean_squared_error(mlpr.predict(X_test), y_test)
holdout_mse = mean_squared_error(mlpr.predict(X_holdout), y_holdout)
print('Artifical Neural Network Training (MSE) is: ', train_mse)
print("Artifical Neural Network Training R2:", mlpr.score(X_train, y_train))
print('Artifical Neural Network Testing (MSE) is: ', test_mse)
print("Artifical Neural Network Testing R2:", mlpr.score(X_test, y_test))
print('Artifical Neural Network Holdout (MSE) is: ', holdout_mse)
print("Artifical Neural Network Holdout R2:", mlpr.score(X_holdout, y_holdout))

Artifical Neural Network Training (MSE) is:  63.10376600525278
Artifical Neural Network Training R2: 0.5873542528626248
Artifical Neural Network Testing (MSE) is:  63.32527066782309
Artifical Neural Network Testing R2: 0.5861736998806566
Artifical Neural Network Holdout (MSE) is:  68.58328035889825
Artifical Neural Network Holdout R2: 0.5864603918812784


In [ ]:
# Use GridSearch to find the best combination of model hyperparameters
mlpr = MLPRegressor()
param_grid = {"activation":['logistic', 'relu', 'tanh'],
             "hidden_layer_sizes": [5,10,20,100],
             "max_iter": [200, 2000, 4000],
             "solver": ['sgd', 'adam']}
gs_ann = GridSearchCV(mlpr, param_grid=param_grid, scoring='neg_mean_squared_error', cv= 5, n_jobs= -1)
gs_ann = gs_ann.fit(X_train, y_train)
print (gs_ann.best_score_)
print (gs_ann.best_params_)

##### Random Forest Regressor

In [107]:
rfr = RandomForestRegressor()
rfr.fit(X_train,y_train)
train_mse = mean_squared_error(rfr.predict(X_train), y_train)
test_mse = mean_squared_error(rfr.predict(X_test), y_test)
holdout_mse = mean_squared_error(rfr.predict(X_holdout), y_holdout)
print('Random Forest Training mean squared error (MSE) is: ', train_mse)
print('Random Forest Training R2:', rfr.score(X_train,y_train))
print('Random Forest Testing mean squared error (MSE) is: ', test_mse)
print('Random Forest Testing R2:', rfr.score(X_test,y_test))
print('Random Forest Holdout mean squared error (MSE) is: ', holdout_mse)
print('Random Forest Holdout R2:', rfr.score(X_holdout,y_holdout))

Random Forest Training mean squared error (MSE) is:  53.158771685309866
Random Forest Training R2: 0.6523861815606393
Random Forest Testing mean squared error (MSE) is:  66.67262027799505
Random Forest Testing R2: 0.5642989997842345
Random Forest Holdout mean squared error (MSE) is:  69.76348195449094
Random Forest Holdout R2: 0.5793440786517499


In [ ]:
rfr = RandomForestRegressor()
param_grid = { "criterion" : ["absolute_error", "squared_error"], 
              "max_depth": [2, 4, 6, 8], 
              "min_samples_leaf" : [1, 5, 10], 
              "n_estimators": [10, 20, 50, 100]}

gs_rf = GridSearchCV(estimator=rfr, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)

gs_rf = gs_rf.fit(X_train_test, y_train_test)

print(gs_rf.best_score_)
print(gs_rf.best_params_)

##### Gradient Boost Regressor

In [108]:
gbr = GradientBoostingRegressor()
gbr.fit(X_train,y_train)
train_mse = mean_squared_error(gbr.predict(X_train), y_train)
test_mse = mean_squared_error(gbr.predict(X_test), y_test)
holdout_mse = mean_squared_error(gbr.predict(X_holdout), y_holdout)
print('Gradient Boosting Training mean squared error (MSE) is: ', train_mse)
print("Gradient Boosting Training R2:", gbr.score(X_train,y_train))
print('Gradient Boosting Testing mean squared error (MSE) is: ', test_mse)
print("Gradient Boosting Testing R2:", gbr.score(X_test,y_test))
print('Gradient Boosting Holdout mean squared error (MSE) is: ', holdout_mse)
print("Gradient Boosting Holdout R2:", gbr.score(X_holdout,y_holdout))

Gradient Boosting Training mean squared error (MSE) is:  62.39836213497974
Gradient Boosting Training R2: 0.5919670030280932
Gradient Boosting Testing mean squared error (MSE) is:  62.68061234689876
Gradient Boosting Testing R2: 0.5903864977885986
Gradient Boosting Holdout mean squared error (MSE) is:  68.21767257800062
Gradient Boosting Holdout R2: 0.5886649131238664


In [ ]:
gbr = GradientBoostingRegressor()
param_grid = {
    'n_estimators': [50, 100, 150],            # Number of boosting stages to be run
    'learning_rate': [0.1, 0.05, 0.01],        # Step size shrinks the contribution of each tree
    'max_depth': [3, 5, 7]                   # Maximum depth of the individual trees
                
}
gs_gbr = GridSearchCV(estimator=gbr, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
gs_gbr = gs_gbr.fit(X_train, y_train)

print(gs_gbr.best_score_)
print(gs_gbr.best_params_)

##### XGBoost Regressor

In [101]:
xgbr = XGBRegressor()
xgbr.fit(X_train,y_train)
train_mse = mean_squared_error(xgbr.predict(X_train), y_train)
test_mse = mean_squared_error(xgbr.predict(X_test), y_test)
holdout_mse = mean_squared_error(xgbr.predict(X_holdout), y_holdout)
print('XGBoost Regressor Training mean squared error (MSE) is: ', train_mse)
print("XGBoost Regressor Training R2", xgbr.score(X_train,y_train))
print('XGBoost Regressor Testing mean squared error (MSE) is: ', test_mse)
print("XGBoost Regressor Testing R2:", xgbr.score(X_test,y_test))
print('XGBoost Regressor Holdout mean squared error (MSE) is: ', holdout_mse)
print("XGBoost Regressor Holdout R2:", xgbr.score(X_holdout,y_holdout))

XGBoost Regressor Training mean squared error (MSE) is:  59.1113628865791
XGBoost Regressor Training R2 0.6134612235249075
XGBoost Regressor Testing mean squared error (MSE) is:  62.04497903485528
XGBoost Regressor Testing R2: 0.5945403178825601
XGBoost Regressor Holdout mean squared error (MSE) is:  69.58446734307172
XGBoost Regressor Holdout R2: 0.5804234908913866


In [103]:
xgbr = XGBRegressor()
param_grid = {"n_estimators": [50, 100, 150],
             "learning_rate":[0.001, 0.01, 0.1]}

gs_xgb = GridSearchCV(estimator=xgbr, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)

gs_xgb = gs_xgb.fit(X_train_test,y_train_test)

print(gs_xgb.best_score_)
print(gs_xgb.best_params_)

-62.92325512273665
{'learning_rate': 0.1, 'n_estimators': 50}


In [104]:
xgbr = XGBRegressor(n_estimators=50, learning_rate = 0.1)
xgbr.fit(X_train,y_train)
train_mse = mean_squared_error(xgbr.predict(X_train), y_train)
test_mse = mean_squared_error(xgbr.predict(X_test), y_test)
holdout_mse = mean_squared_error(xgbr.predict(X_holdout), y_holdout)
print('XGBoost Regressor Training mean squared error (MSE) is: ', train_mse)
print("XGBoost Regressor Training R2", xgbr.score(X_train,y_train))
print('XGBoost Regressor Testing mean squared error (MSE) is: ', test_mse)
print("XGBoost Regressor Testing R2:", xgbr.score(X_test,y_test))
print('XGBoost Regressor Holdout mean squared error (MSE) is: ', holdout_mse)
print("XGBoost Regressor Holdout R2:", xgbr.score(X_holdout,y_holdout))

XGBoost Regressor Training mean squared error (MSE) is:  61.65810607582289
XGBoost Regressor Training R2 0.5968076573018544
XGBoost Regressor Testing mean squared error (MSE) is:  62.411668674321426
XGBoost Regressor Testing R2: 0.5921440262411337
XGBoost Regressor Holdout mean squared error (MSE) is:  68.23728097699814
XGBoost Regressor Holdout R2: 0.588546679501986


##### AdaBoost Regressor

In [109]:
abd = AdaBoostRegressor()
abd.fit(X_train, y_train)
train_mse = mean_squared_error(abd.predict(X_train), y_train)
test_mse = mean_squared_error(abd.predict(X_test), y_test)
holdout_mse = mean_squared_error(abd.predict(X_holdout), y_holdout)
print('AdaBoostRegressor Training mean squared error (MSE) is: ', train_mse)
print("AdaBoostRegressor Training R2", abd.score(X_train,y_train))
print('AdaBoostRegressor Testing mean squared error (MSE) is: ', test_mse)
print("AdaBoostRegressor Testing R2:", abd.score(X_test,y_test))
print('AdaBoostRegressor Holdout mean squared error (MSE) is: ', holdout_mse)
print("AdaBoostRegressor Holdout R2:", abd.score(X_holdout,y_holdout))

AdaBoostRegressor Training mean squared error (MSE) is:  249.32948438077096
AdaBoostRegressor Training R2 -0.6304058834953781
AdaBoostRegressor Testing mean squared error (MSE) is:  248.28054966998485
AdaBoostRegressor Testing R2: -0.6224963616891988
AdaBoostRegressor Holdout mean squared error (MSE) is:  192.1637721996066
AdaBoostRegressor Holdout R2: -0.15869831005728807


In [ ]:
abd = AdaBoostRegressor()
param_grid = {"n_estimators": [10, 20, 50, 100],
             "learning_rate":[0.0001, 0.001, 0.01, 0.1, 1.0]}

gs_abd = GridSearchCV(estimator=abd, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)

gs_abd = gs_abd.fit(X_train_test, y_train_test)

print(gs_abd.best_score_)
print(gs_abd.best_params_)

In [ ]:


# Get feature importance scores from the model (using coefficients)
feature_importance = model_linear_regression.coef_

# Create a DataFrame to display the feature importance
importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importance})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Display the feature importance
print(importance_df)

In [ ]:


# Get feature importance scores from the model (using coefficients)
feature_importance = model_linear_regression.coef_

# Create a DataFrame to display the feature importance
importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importance})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Display the feature importance
print(importance_df)

In [ ]:
feat_importances = pd.Series(dtreg.feature_importances_, index=X_train.columns)
feat_importances.nlargest(33).plot(kind='barh')

### GET OUT A SMALLER DATA SAMPLE

In [ ]:
# sample = 10
# sample_data_hold = bottom_10_df.sample(frac = sample / 100, random_state = 2)

In [ ]:
# sample = 10
# sample_data_testing = testing_data.sample(frac = sample / 100, random_state = 2)

In [ ]:
# sample_data_hold.info()

In [ ]:
# sample_data_testing.info()

In [ ]:
# # For Validation Data
# X_holdout_sam = sample_data_hold.drop(['TOTAL_SALE_PER_ITEM_PER_SHIFT'], axis=1)
# y_holdout_sam = sample_data_hold['TOTAL_SALE_PER_ITEM_PER_SHIFT']

In [ ]:
# # 90% for train and testing data 
# X_train_test_sam = sample_data_testing.drop(['TOTAL_SALE_PER_ITEM_PER_SHIFT'], axis=1)
# y_train_test_sam = sample_data_testing['TOTAL_SALE_PER_ITEM_PER_SHIFT']

In [ ]:
# X_train_sam, X_test_sam, y_train_sam, y_test_sam = train_test_split(X_train_test_sam, y_train_test_sam, test_size=0.3, random_state=2)

In [ ]:
# print(X_train_sam.info())
# print(X_test_sam.info())

In [ ]:
# #Standardization
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaler.fit(X_train_sam)
# X_train_scaled_sam = scaler.transform(X_train_sam)
# X_test_scaled_sam = scaler.transform(X_test_sam)
# X_train_scaled_sam = pd.DataFrame(X_train_scaled_sam, columns=X_train_sam.columns)
# X_test_scaled_sam = pd.DataFrame(X_test_scaled_sam, columns=X_test_sam.columns)
# display(X_train_scaled_sam.describe())

In [ ]:
# # Base Model
# model_linear_regression = LinearRegression()
# model_linear_regression.fit(X_train_sam, y_train_sam)
# y_pred_holdout = model_linear_regression.predict(X_holdout_sam)
# mse_holdout = mean_squared_error(y_holdout_sam, y_pred_holdout)

# print('train_mse:', mean_squared_error(y_train_sam, model_linear_regression.predict(X_train_sam)), 
#       '\n test_mse:', mean_squared_error(y_test_sam, model_linear_regression.predict(X_test_sam)))
# print('train_r-sq:', r2_score(y_train_sam, model_linear_regression.predict(X_train_sam)), 
#       '\n test_r-sq:', r2_score(y_test_sam, model_linear_regression.predict(X_test_sam)))

# print('holdout_mse:', mean_squared_error(y_holdout_sam, model_linear_regression.predict(X_holdout_sam)), 
#       '\n holdout-sq:',  r2_score(y_holdout_sam, model_linear_regression.predict(X_holdout_sam)))

In [ ]:
# rfr = RandomForestRegressor()
# rfr.fit(X_train_sam,y_train_sam)
# train_mse = mean_squared_error(rfr.predict(X_train_sam), y_train_sam)
# test_mse = mean_squared_error(rfr.predict(X_test_sam), y_test_sam)
# holdout_mse = mean_squared_error(rfr.predict(X_holdout_sam), y_holdout_sam)
# print('Random Forest Training mean squared error (MSE) is: ', train_mse)
# print('Random Forest Training R2:', rfr.score(X_train_sam,y_train_sam))
# print('Random Forest Testing mean squared error (MSE) is: ', test_mse)
# print('Random Forest Testing R2:', rfr.score(X_test_sam,y_test_sam))
# print('Random Forest Holdout mean squared error (MSE) is: ', holdout_mse)
# print('Random Forest Holdout R2:', rfr.score(X_holdout_sam,y_holdout_sam))

In [ ]:
# rfr = RandomForestRegressor()
# param_grid = { "criterion" : ["absolute_error", "squared_error"], 
#               "max_depth": [2, 4, 6, 8], 
#               "min_samples_leaf" : [1, 5, 10], 
#               "n_estimators": [10, 20, 50, 100]}

# gs_rf = GridSearchCV(estimator=rfr, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)

# gs_rf = gs_rf.fit(X_train_sam, y_train_sam)

# print(gs_rf.best_score_)
# print(gs_rf.best_params_)